<a href="https://colab.research.google.com/github/JulianCarax01/Fondamenti-web-app/blob/main/DijkstraSerialCPPMerge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile dijkstra_serialMerge.cu
#include <iostream>
#include <vector>
#include <cstdlib>
#include <climits>
#include <cuda.h>
#include <curand_kernel.h>

#define GRIDSIZE 1024
#define BLOCKSIZE 1024

// Kernel CUDA per aggiornare le distanze
__global__ void cudaNodeRelax(int *graph, int *distances, char *visited, int n, int currentNode) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x; // Stabiliamo l'indice generale del thread tramite le formule viste a lezione

    // Verifichiamo che l'indice sia inferiore al numero totale dei nodi e che il nodo non sia già stato visitato,
    // inoltre facciamo un check per verificare che la distanza tra due nodi non sia zero
    if (idx < n && !visited[idx] && graph[currentNode * n + idx] > 0) {
        int newDist = distances[currentNode] + graph[currentNode * n + idx];
        atomicMin(&distances[idx], newDist); // Aggiorniamo la distanza usando atomicMin per evitare condizioni di gara
    }
    __syncthreads();
}

__global__ void findClosestNodeCUDA(int* d_graph, int d_distances[], char* d_visited, int n){
  //calcolo di thread id per unrolling
  //int tid = blockIdx.x * blockDim.x + threadIdx.x;
  for (int i = 0; i < n; ++i) {

        // Troviamo il nodo non visitato con distanza minima
        int minDist = INT_MAX;
        int currentNode = -1;
        for (int j = 0; j < n; j++) {
            // Controlliamo che il nodo non sia stato visitato e che la distanza sia inferiore a quella nota
            if (!d_visited[j] && d_distances[j] < minDist) {
                minDist = d_distances[j];
                currentNode = j;
            }
        }

        if (currentNode == -1) break; // Se non ci sono più nodi raggiungibili
        d_visited[currentNode] = 1;
/*
        // Aggiorna le distanze in parallelo
        cudaMemcpy(d_distances, distances.data(), n * sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(d_visited, visited.data(), n * sizeof(char), cudaMemcpyHostToDevice);*/

        cudaNodeRelax<<<64, 64>>>(d_graph, d_distances, d_visited, n, currentNode);
        __syncthreads();
/*
        cudaMemcpy(distances.data(), d_distances, n * sizeof(int), cudaMemcpyDeviceToHost);*/
    }
}

// Funzione host per eseguire Dijkstra in CUDA
void dijkstraCUDA(int *graph, int n, int start) {
    // Allocazione memoria su GPU
    int *d_graph; //puntatore memorua gpu
    int *d_distances;
    char *d_visited; // Usiamo un tipo char per verificare che il nodo sia già stato verificato o no (non riesco a farlo con il bool)

    // Utilizziamo la malloc per andare ad allocare sulla memoria
    cudaMalloc((void **)&d_graph, n * n * sizeof(int));
    cudaMalloc((void **)&d_distances, n * sizeof(int));
    cudaMalloc((void **)&d_visited, n * sizeof(char));

    // Inizializza distanze e visited
    std::vector<int> distances(n, INT_MAX);
    std::vector<char> visited(n, 0);
    distances[start] = 0;

    // Copia grafico su GPU
    cudaMemcpy(d_graph, graph, n * n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_distances, distances.data(), n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_visited, visited.data(), n * sizeof(char), cudaMemcpyHostToDevice); // Cambiato da bool a char

    // Esegui Dijkstra iterativamente

    findClosestNodeCUDA <<<1024, 1024>>>(d_graph, d_distances, d_visited, n);

    // Libera memoria GPU
    cudaFree(d_graph);
    cudaFree(d_distances);
    cudaFree(d_visited);

      /* // Stampa la matrice con "X" sulla diagonale
    std::cout << "Matrice del grafo:\n";
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            if (i == j) {
                std::cout << "X "; // Stampa "X" sulla diagonale
            } else {
                std::cout << graph[i * n + j] << " ";
            }
        }
        std::cout << "\n";
    }

    // Stampa solo le distanze alla fine
    std::cout << "Distanze dal nodo iniziale:\n";
    for (int i = 0; i < n; ++i) {
        if (distances[i] == INT_MAX) {
            std::cout << "Nodo " << i << ": NON ESISTE COLLEGAMENTO TRA I NODI\n";
        } else {
            std::cout << "Nodo " << i << ": " << distances[i] << "\n";
        }
    } */

}



__global__ void generateGraphKernel(int *graph, int n, int minWeight, int maxWeight, unsigned int seed) {
    int row_start = blockIdx.y * 4; // Prima riga gestita dal blocco (4 righe consecutive)
    int col = blockIdx.x * blockDim.x + threadIdx.x; // Colonna gestita dal thread nel blocco

    // Dichiarazione degli stati casuali per 4 righe
    curandState state[4];

    // Inizializzazione dei generatori casuali per le 4 righe
    if (row_start < n) curand_init(seed + row_start, 0, 0, &state[0]);
    if (row_start + 1 < n) curand_init(seed + row_start + 1, 0, 0, &state[1]);
    if (row_start + 2 < n) curand_init(seed + row_start + 2, 0, 0, &state[2]);
    if (row_start + 3 < n) curand_init(seed + row_start + 3, 0, 0, &state[3]);

    // Gestione esplicita di ogni riga
    if (row_start < n && col < n) {
        // Riga 0
        if (row_start < col) {
            int zero_generator = curand(&state[0]) % 100 + 1;
            int weight = (zero_generator <= 40)
                         ? curand(&state[0]) % (maxWeight - minWeight + 1) + minWeight
                         : 0;
            graph[row_start * n + col] = weight;
            graph[col * n + row_start] = weight;
        }
    }

    if (row_start + 1 < n && col < n) {
        // Riga 1
        if (row_start + 1 < col) {
            int zero_generator = curand(&state[1]) % 100 + 1;
            int weight = (zero_generator <= 40)
                         ? curand(&state[1]) % (maxWeight - minWeight + 1) + minWeight
                         : 0;
            graph[(row_start + 1) * n + col] = weight;
            graph[col * n + (row_start + 1)] = weight;
        }
    }

    if (row_start + 2 < n && col < n) {
        // Riga 2
        if (row_start + 2 < col) {
            int zero_generator = curand(&state[2]) % 100 + 1;
            int weight = (zero_generator <= 40)
                         ? curand(&state[2]) % (maxWeight - minWeight + 1) + minWeight
                         : 0;
            graph[(row_start + 2) * n + col] = weight;
            graph[col * n + (row_start + 2)] = weight;
        }
    }

    if (row_start + 3 < n && col < n) {
        // Riga 3
        if (row_start + 3 < col) {
            int zero_generator = curand(&state[3]) % 100 + 1;
            int weight = (zero_generator <= 40)
                         ? curand(&state[3]) % (maxWeight - minWeight + 1) + minWeight
                         : 0;
            graph[(row_start + 3) * n + col] = weight;
            graph[col * n + (row_start + 3)] = weight;
        }
    }

    // Assicura che la diagonale sia sempre 0
    if (col == row_start && row_start < n) {
        graph[row_start * n + row_start] = 0;
    }
    if (col == row_start + 1 && row_start + 1 < n) {
        graph[(row_start + 1) * n + (row_start + 1)] = 0;
    }
    if (col == row_start + 2 && row_start + 2 < n) {
        graph[(row_start + 2) * n + (row_start + 2)] = 0;
    }
    if (col == row_start + 3 && row_start + 3 < n) {
        graph[(row_start + 3) * n + (row_start + 3)] = 0;
    }
}



void generateGraphCUDA(int *graph, int n, int minWeight, int maxWeight) {
    int *d_graph;
    size_t size = n * n * sizeof(int);

    // Allocazione della memoria sulla GPU
    if (cudaMalloc(&d_graph, size) != cudaSuccess) {
        std::cerr << "Errore nell'allocazione della memoria sulla GPU." << std::endl;
        return;
    }
    if (cudaMemset(d_graph, 0, size) != cudaSuccess) {
        std::cerr << "Errore durante l'inizializzazione della memoria sulla GPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    const int blockSize = 256; // Numero di thread per blocco lungo l'asse x //Marco dobbiamo testare qui daje ROMA
dim3 threadsPerBlock(blockSize); // Blocchi 1D (solo sull'asse x)
dim3 blocksPerGrid((n + blockSize - 1) / blockSize, n); // Griglia 2D


    // Lancio del kernel
generateGraphKernel<<<blocksPerGrid, threadsPerBlock>>>(d_graph, n, minWeight, maxWeight, time(NULL));
    if (cudaDeviceSynchronize() != cudaSuccess) {
        std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Copia i dati dalla GPU alla CPU
    if (cudaMemcpy(graph, d_graph, size, cudaMemcpyDeviceToHost) != cudaSuccess) {
        std::cerr << "Errore nella copia dei dati dalla GPU alla CPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Libera memoria sulla GPU
    cudaFree(d_graph);
}


int main() {
    int n = 1024; // Numero di nodi
    int start = 0; // Nodo iniziale
    int minWeight = 1;
    int maxWeight = 5000;

    // Genera il grafo casuale
    std::vector<int> graph(n * n);
    generateGraphCUDA(graph.data(), n, minWeight, maxWeight);

    dijkstraCUDA(graph.data(), n, start);

    return 0;
}









Writing dijkstra_serialMerge.cu


In [ ]:
!ncu --mode launch-and-attach -o profile --target-processes all --nvtx --call-stack --section ComputeWorkloadAnalysis --section MemoryWorkloadAnalysis -f ./dijkstra_serialMerge.o


==PROF== Connected to process 32883 (/content/dijkstra_serialMerge.o)
==PROF== Profiling "generateGraphKernel" - 0: 0%....50%....100% - 8 passes
==PROF== Profiling "findClosestNodeCUDA" - 1: 0%....50%....100% - 8 passes
==PROF== Disconnected from process 32883
==PROF== Report: /content/profile.ncu-rep


In [ ]:
!nvcc -rdc=true dijkstra_serialMerge.cu -lcudadevrt -o dijkstra_serialMerge.o

In [ ]:
!ncu --target-processes=all ./dijkstra_serialMerge.o

==PROF== Connected to process 33243 (/content/dijkstra_serialMerge.o)
==PROF== Profiling "generateGraphKernel" - 0: 0%....50%....100% - 8 passes
==PROF== Profiling "findClosestNodeCUDA" - 1: 0%....50%....100% - 8 passes
==PROF== Disconnected from process 33243
[33243] dijkstra_serialMerge.o@127.0.0.1
  generateGraphKernel(int *, int, int, int, unsigned int) (64, 64, 1)x(16, 16, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- -------------
    Metric Name               Metric Unit  Metric Value
    ----------------------- ------------- -------------
    DRAM Frequency          cycle/nsecond          4.94
    SM Frequency            cycle/usecond        578.23
    Elapsed Cycles                  cycle    31,092,761
    Memory Throughput                   %         47.04
    DRAM Throughput                     %          4.21
    Duration                      msecond         53.77
    L1/TEX Cache Throughput   